<a href="https://colab.research.google.com/github/gretiere545/tradunion/blob/master/tupy_country_x_language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table des relations Pays x Langues

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option("display.width",1000)

In [ ]:
# Ce bout de code pour pouvoir downloader des fichiers sur le drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# !ls "/content/drive/My Drive/Trad-Union/Cours/Géographie des langues"

## Intégration des documents google drive dans pandas

In [47]:
from google.colab import auth
auth.authenticate_user()
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

### importation du workbook **tu-wb-1** et chargement des tables

*   t_ctry
*   t_lang_x_ctry

dans des dataframes

In [48]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1bMnE2PYl6niMHV7_bXJqTR9GJRLyitW1ANjFvemf8Q4')
t_ctry = wb.worksheet('t_ctry')
data_t_ctry = t_ctry.get_all_values()
df_ctry = pd.DataFrame(data_t_ctry[1:], columns=data_t_ctry[0])

In [49]:
t_lang_x_ctry = wb.worksheet('t_lang_X_ctry')
data_t_lang_x_ctry = t_lang_x_ctry.get_all_values()
df_lang_x_ctry = pd.DataFrame(data_t_lang_x_ctry[1:], columns=data_t_lang_x_ctry[0])

#### suppression des valeurs nulles et tri par *pays*

In [50]:
df_lang_x_ctry = df_lang_x_ctry.dropna()
df_lang_x_ctry.sort_values(by=["alpha-2_code","wals_code"], ascending=True, inplace=True)


#### redéfinition de la clé par *pays-langue*

In [51]:
df_lang_x_ctry["key_id"]=df_lang_x_ctry.apply(lambda x: x["alpha-2_code"]+"-"+x["wals_code"], axis=1)


#### remplacement des cellules vides par NAN puis supression des lignes concernées




In [52]:
df_lang_x_ctry["alpha-2_code"]=df_lang_x_ctry["alpha-2_code"].apply(lambda x: x.strip()).replace('', np.nan)
df_lang_x_ctry = df_lang_x_ctry.loc[~df_lang_x_ctry["alpha-2_code"].isnull()]

In [55]:
df_lang_x_ctry = df_lang_x_ctry[["key_id",	"alpha-2_code","wals_code",	"rank",	"speakers",	"comment"]]

In [57]:
export_sheet = wb.worksheet('t_temp')
set_with_dataframe(export_sheet, df_lang_x_ctry)
